In [1]:
import PyTorch
import PyTorchHelpers
import numpy as np
from numpy import genfromtxt
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import os
import subprocess as sp
import io
import base64
import top_down_viewer as td
from IPython.display import HTML

from pylab import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from matplotlib import colors 
from matplotlib import patches
from PIL import Image as IM
from IPython.display import display
import glob
import os.path

PATCH_W = 20
PATCH_H = 20

def to_img(v, path=None, patch_w=20, patch_h=20, vmin=0, vmax=255):
    v_img = np.zeros((v.shape[0], v.shape[1], 3), np.uint8)
    for y in range(0, v.shape[0]):
        for x in range(0, v.shape[1]):
            # viridis, inferno
            v_img[y][x][0] = plt.cm.inferno(int(float(v[y][x]-vmin)/(vmax-vmin)*255))[0] * 255
            v_img[y][x][1] = plt.cm.inferno(int(float(v[y][x]-vmin)/(vmax-vmin)*255))[1] * 255
            v_img[y][x][2] = plt.cm.inferno(int(float(v[y][x]-vmin)/(vmax-vmin)*255))[2] * 255
    v_img = IM.fromarray(v_img, "RGB")
    v_img = v_img.resize((v.shape[1]*patch_w, v.shape[0]*patch_h), IM.NEAREST)
    if path:
        filename = path + ".png" 
        v_img.save(filename)
    return v_img
    
    
def read_csv(filename, dtype=float):
    try:
        return np.genfromtxt(filename, delimiter=",", dtype=dtype)
    except IOError:
        return None
        #print(filename + " not found.")
        
def play_and_save_rawdata(d, path=None, viewer=None):
    if path:
        sp.call(['rm', '-rf', path])
        sp.call(['mkdir', '-p', path])
    
    d.new_game()    
    
    # Instructions
    goal_file = open(path + "/goal.txt", "w")
    subtask_file = None
    
    goal = d.goal_text()
    for i in range(0, len(goal)):
        goal_file.write(goal[i+1] + "\n")
    goal_file.close()
    
    max_step = np.int(d.max_step())
    task_idx = np.zeros(max_step, np.uint8)
    act = np.zeros(max_step, np.uint8)
    update = np.ones(max_step, np.float)
    update_val = np.ones(max_step, np.float)
    sub_term = np.ones(max_step, np.uint8)
    term_pred = np.ones(max_step, np.float)
    pointer = np.zeros((max_step, 20), np.float)
    shift = np.zeros((max_step, 3), np.float)
    register = np.zeros((max_step, 20), np.float)
    reward = np.zeros(max_step, np.float)
    
    sub_giveup = np.zeros(max_step, np.uint8)
    giveup = np.ones(max_step, np.uint8)
    giveup_prob = np.zeros(max_step, np.float)
    
    h_norm1 = np.zeros(max_step, np.float)
    h_norm2 = np.zeros(max_step, np.float)
    
    term = False
    t = 0
    
    obj_sum = 0
    if viewer: 
        ret = d.get_metadata()
        viewer.draw_topology(ret['topology'].asNumpyTensor())
        
    while t < max_step:
        if viewer: 
            ret = d.get_metadata()
            new_sum = ret['objects'].asNumpyTensor().sum()
            changed = new_sum != obj_sum
            viewer.draw_objects(ret['objects'].asNumpyTensor(), changed)
            obj_sum = new_sum
            img = viewer.update_frame(ret['pos_x'], ret['pos_y'], ret['yaw'])
        else:
            img = Image.fromarray((d.map_image().asNumpyTensor().transpose(1,2,0)
                                   * 255).astype(np.uint8), 'RGB').convert('RGBA')
        
        obs = Image.fromarray((d.observation().asNumpyTensor().transpose(1,2,0)
                           * 255).astype(np.uint8), 'RGB').convert('RGBA')
        task_idx[t] = d.task_idx()
        active = d.active()
        width, height = obs.size
        if width != 64:
            obs_resized = np.asarray(obs.resize((64, 64), Image.ANTIALIAS).convert('RGB'), dtype=np.float32)
            obs_resized = obs_resized.astype(float).transpose(2,0,1) / 255.0
            d.update_observation(obs_resized)
        else:
            d.update_observation()
            
        ret = d.update()    
        if active == 1.0:
            # Action
            act[t] = np.int(ret['act'])
            reward[t] = np.float(ret['reward'])
            
            # Sub-task
            sub_task_str = d.sub_task()
            if sub_task_str != "":    
                if subtask_file is None:
                    subtask_file = open(path + "/subtask.txt", "w")
                subtask_file.write(sub_task_str + "\n")
            if d.update_decision() is not None: 
                update[t] = d.update_decision()
            if d.memory_pointer() is not None:
                pointer[t] = d.memory_pointer().asNumpyTensor()
            if d.shift() is not None:
                shift[t] = d.shift().asNumpyTensor()
                
            if d.register() is not None:
                register[t] = d.register().asNumpyTensor()
            
            if d.sub_term() is not None:
                sub_term[t] = np.int(d.sub_term())
                
            if d.giveup() is not None:
                giveup[t] = np.int(d.giveup())
                giveup_prob[t] = d.giveup_prob()
                
            if d.sub_giveup() is not None:
                sub_giveup[t] = np.int(d.sub_giveup())
                
            if d.h_norm1() is not None:
                h_norm1[t] = d.h_norm1()
            
            if d.h_norm2() is not None:
                h_norm2[t] = d.h_norm2()
                
            if d.update_value() is not None:
                update_val[t] = d.update_value()
            
        if d.term() is not None:
            term_pred[t] = np.float(d.term())
        if path:
            file_name = "%s/map_%03d.png" % (path, t)
            img.convert('RGB').save(file_name)
            file_name = "%s/obs_%03d.png" % (path, t)
            obs.convert('RGB').save(file_name)
        
        t = t + 1
        if d.active() != 1.0 and not term:
            term = True
        elif d.active() == 1.0:
            term = False
        else:
            break    
    
    task_idx.resize(t)
    act.resize(t-1)
    update.resize(t-1)
    pointer.resize(t-1, 20)
    register.resize(t-1, 20)
    shift.resize(t-1, 3)
    sub_term.resize(t-1)
    term_pred.resize(t)
    
    h_norm1.resize(t-1)
    h_norm2.resize(t-1)
    update_val.resize(t-1)
    
    np.savetxt(path + "/task_idx.csv", task_idx, delimiter=",", fmt='%i')
    np.savetxt(path + "/act.csv", act, delimiter=",", fmt='%i')
    np.savetxt(path + "/reward.csv", reward, delimiter=",", fmt="%.2f")
    if subtask_file:    
        subtask_file.close()
    if d.memory_pointer() is not None:
        np.savetxt(path + "/pointer.csv", pointer, delimiter=",", fmt='%.3f')
    if d.update_decision() is not None:
        np.savetxt(path + "/update.csv", update, delimiter=",", fmt='%.3f')            
    if d.shift() is not None:
        np.savetxt(path + "/shift.csv", shift, delimiter=",", fmt='%.3f')
    if d.register() is not None:
        np.savetxt(path + "/register.csv", register, delimiter=",", fmt='%.3f')
    if d.sub_term() is not None:
        np.savetxt(path + "/sub_term.csv", sub_term, delimiter=",", fmt='%i')
    if d.term() is not None:
        np.savetxt(path + "/term.csv", term_pred, delimiter=",", fmt='%.3f')
        
    if d.giveup() is not None:
        np.savetxt(path + "/giveup.csv", giveup, delimiter=",", fmt='%i')
        np.savetxt(path + "/giveup_prob.csv", giveup_prob, delimiter=",", fmt='%.3f')
        
    if d.sub_giveup() is not None:
        np.savetxt(path + "/sub_giveup.csv", sub_giveup, delimiter=",", fmt='%i')
        
    if d.h_norm1() is not None:
        np.savetxt(path + "/h_norm1.csv", h_norm1, delimiter=",", fmt='%.3f')
        
    if d.h_norm2() is not None:
        np.savetxt(path + "/h_norm2.csv", h_norm2, delimiter=",", fmt='%.3f')
        
    if d.update_value() is not None:
        np.savetxt(path + "/update_val.csv", update_val, delimiter=",", fmt='%.3f')
    
def visualize_and_save(path, img_size=256):
    text_width = 300
    font_size = 16
    inst_size = 13
    default_color = (0, 0, 0, 255)
    highlight = (130, 200, 255, 255)
    fnt_normal = ImageFont.truetype('resource/OpenSans-Semibold.ttf', font_size)
    fnt_bold = ImageFont.truetype('resource/OpenSans-ExtraBold.ttf', font_size)
    fnt_inst_normal = ImageFont.truetype('resource/OpenSans-Semibold.ttf', inst_size)
    fnt_inst_bold = ImageFont.truetype('resource/OpenSans-ExtraBold.ttf', inst_size)
    act_name = ['look_left', 'look_right', 'look_up', 'look_down', 'move_forward',
               'move backward', 'stop', 'pick up', 'transform']

    task_idx = read_csv(path + "/task_idx.csv", int)
    act = read_csv(path + "/act.csv", int)
    reward = read_csv(path + "/reward.csv", float)
    update = read_csv(path + "/update.csv", float)
    pointer = read_csv(path + "/pointer.csv", float)
    shift = read_csv(path + "/shift.csv", float)
    register = read_csv(path + "/register.csv", float)
    sub_term = read_csv(path + "/sub_term.csv", int)
    term_pred = read_csv(path + "/term.csv", float)
    
    sub_giveup = read_csv(path + "/sub_giveup.csv", int)
    giveup = read_csv(path + "/giveup.csv", int)
    giveup_prob = read_csv(path + "/giveup_prob.csv", float)
    update_val = read_csv(path + "/update_val.csv", float)
    
    with open(path + "/goal.txt") as f:
        goal_text = f.readlines()
        
    try:
        with open(path + "/subtask.txt") as f:
            subtask_text = f.readlines()
    except IOError:
        subtask_text = None
        #print(path + "/subtask.txt" + " not found.") 
    
    display_img = Image.new('RGBA', (2*img_size + text_width, img_size), (255, 255, 255, 0))
    draw = ImageDraw.Draw(display_img)
    
    max_step = task_idx.shape[0]
    for t in range(0, max_step):
        img = Image.open("%s/map_%03d.png" % (path, t))
        obs = Image.open("%s/obs_%03d.png" % (path, t))
        obs = obs.resize((img_size, img_size), Image.ANTIALIAS)
        
        # World image
        display_img.paste(obs, (0,0))
        display_img.paste(img, (img_size,0))
        draw.rectangle([2*img_size, 0, 2*img_size + text_width, img_size], (255, 255, 255, 255))
        h_off = 0
        w_off = 2*img_size + 10
        
        #Step
        #draw.text((w_off, h_off), 'Step: ' + str(t+1), font=fnt_normal, fill=default_color)
        #h_off = h_off + font_size + 5
        
        if t < max_step - 1:            
            # Action
            #try:
            #    act_str = 'Action: ' + act_name[np.int(act[t]) - 1]
                #reward_str = 'Reward: ' + ("%.2f" % reward[t])
            #except:
            #    act_str = 'Action: ' + act_name[np.int(act) - 1]
                #reward_str = 'Reward: ' + ("%.2f" % reward)
                
            #draw.text((w_off, h_off), act_str, font=fnt_normal, fill=default_color)
            #h_off = h_off + font_size + 5
            
#             draw.text((w_off, h_off), reward_str, font=fnt_normal, fill=default_color)
#             h_off = h_off + font_size + 5

            # Sub-task
            if subtask_text is not None:
                fnt = fnt_normal
                draw.text((w_off, h_off), 
                          "Subtask: ",
                          font=fnt, 
                          fill=default_color)
                text_size = draw.textsize("Subtask: ", font=fnt_normal)
                #if update is None or update[t] == 1:
                #    fnt = fnt_bold
                
                txt = subtask_text[t].replace("\n", "").replace("enemy", "greenbot")
                if update is not None:
                    txt = txt + '(%.3f)' % update[t]
                if sub_term is not None and sub_term[t] == 1:
                    txt = txt + '(t)'
                
                #if sub_giveup is not None and sub_giveup[t] == 1:
                #    txt = txt + '(f)'
                    
                draw.text((w_off + text_size[0], h_off), 
                      txt, 
                      font=fnt, 
                      fill=default_color)
                h_off = h_off + font_size + 5
              
            
            
            if register is not None:
                r_size = register.shape[1]
                res_vis = (register[t] * 255).clip(0, 255).astype(np.uint8)
                res_vis = res_vis.reshape(1, r_size)
                pcolor(res_vis, cmap=plt.cm.inferno)
                res_w = min(15, int(360/15))
                res_img = to_img(res_vis, patch_h=15, patch_w=res_w)
                display_img.paste(res_img, (w_off, h_off))
                h_off = h_off + 20

            if giveup is not None:
                draw.text((w_off, h_off), 
                          "Giveup: " + ("%.2f" % giveup_prob[t]),
                          font=fnt_normal, 
                          fill=default_color)
                h_off = h_off + font_size + 5
        
        if term_pred is not None:
            draw.text((w_off, h_off), 
                      "Term: " + ("%.3f" % term_pred[t]),
                      font=fnt_normal, 
                      fill=default_color)
            h_off = h_off + font_size + 5
        # Instructions
        h_off = h_off + 10
        h_start = h_off
        for i in range(0, len(goal_text)):
            fnt = fnt_inst_normal
            color = default_color
            
            if i == task_idx[t] - 1:
                fnt = fnt_inst_bold
                            
            if i == 10:
                h_off = h_start
                w_off = w_off + text_width/2
            
            goal_text[i] = goal_text[i].replace("pork", "meat").replace("enemy", "greenbot")
            if pointer is not None and t < max_step - 1: 
                w = pointer[t][i]
                rect_color = (np.int(highlight[0] * w + 255 * (1-w)), 
                              np.int(highlight[1] * w + 255 * (1-w)), 
                              np.int(highlight[2] * w + 255 * (1-w)), 
                              255)
                text_size = draw.textsize(goal_text[i], font=fnt)
                draw.rectangle([w_off, h_off, w_off + text_size[0], h_off + inst_size + 4],
                                fill=rect_color)
                         
            draw.text((w_off, h_off), goal_text[i], font=fnt, fill=color)
            h_off = h_off + inst_size + 5
        
        #display(display_img)
        if path:
            file_name = "%s/%03d.png" % (path, t)
            display_img.convert('RGB').save(file_name)
        
        t = t + 1
    
    if path:
        sp.call(['rm', '-rf', path+'.mp4'])
        sp.call(['ffmpeg', '-r', '2', '-i', path + '/%03d.png', 
                 '-vcodec', 'libx264', '-pix_fmt', 'yuv420p', path+'.mp4'])

def visualize_internal(path):
    update_val = read_csv(path + "/update_val.csv")
    if update_val is not None:
        update_val = (update_val * 255.0).clip(0, 255.0)
        update_val = update_val.reshape((1, update_val.shape[0]))
        to_img(update_val, path + "/update_val")
        display(Image.open(path + "/update_val.png"))
        
    term_v = read_csv(path + "/update.csv")
    if term_v is not None:
        term_v = (term_v * 255.0).clip(0, 255.0)
        term_v = term_v.reshape((1, term_v.shape[0]))
        #pcolor(term_v, cmap=plt.cm.inferno)
        to_img(term_v, path + "/update")

        v = read_csv(path + "/shift.csv").astype(float) * 255
        for i in range(0, v.shape[0]):
            for j in range(0, v.shape[1]):
                v[i, j] -= 255.0 - term_v[0, i]
            
            #if term_v[0, i] != 255:
            #    v[i, :] = 0 
        v = v.clip(0, 255)
        v = v.transpose(1, 0)
        to_img(v, path + "/shift")

        display(Image.open(path + "/update.png"))
        display(Image.open(path + "/shift.png"))
    
    h_norm1 = read_csv(path + "/h_norm1.csv")
    h_norm2 = read_csv(path + "/h_norm2.csv")
    
    if h_norm1 is not None:
        h_norm1 = h_norm1.reshape((1, h_norm1.shape[0]))
        to_img(h_norm1, path + "/h_norm1", vmin=h_norm1.min(), vmax=h_norm1.max())
        
        h_norm2 = h_norm2.reshape((1, h_norm2.shape[0]))
        to_img(h_norm2, path + "/h_norm2", vmin=h_norm2.min(), vmax=h_norm2.max())
        
        display(Image.open(path + "/h_norm1.png"))
        display(Image.open(path + "/h_norm2.png"))
        
    
def demo(game, model, video=None, num_play=1, multitask=False, sub_agent="", img_size=256, 
         counting=False, td_viewer=False, interact=False, open_loop=False, counting2=False):
    d = PyTorchHelpers.load_lua_class('demo.lua', 'Demo')(game, model, multitask, sub_agent, img_size, 1, 
                                                                counting, interact, open_loop, counting2)
    if video:
        sp.call(['rm', '-rf', video])
        sp.call(['mkdir', '-p', video])
        list_file = open(video + '/list.txt', 'w')
    
    if td_viewer:
        viewer = td.TopDownViewer()
        viewer.initialize('Forge/eclipse/tasks/maze/blockTypeInfo.xml', img_size)
    for iter in range(0, num_play):
        video_dir = None
        if video:
            video_dir = '%s/%02d' % (video, iter)
            list_file.write('file %02d.mp4\n' % (iter))
            if td_viewer:
                play_and_save_rawdata(d, video_dir, viewer)
                viewer.reset()
            else:
                play_and_save_rawdata(d, video_dir)
                
            visualize_and_save(video_dir, img_size)
            visualize_internal(video_dir)
    
    if video:
        list_file.close()
        sp.call(['ffmpeg', '-safe', '0', '-f', 'concat', '-i', video + '/list.txt', '-c', 'copy', video + '/video.mp4'])
        sp.call(['rm', video + '/list.txt'])
    
    d.close()
        

In [8]:
game = 'a_count_nc'
model = 'count_min/a_count_minimal_open_rho_init_soft.latest.t7'
video = '../video/det'
num_play = 3

demo(game, model, video=video, num_play=num_play, multitask=False, td_viewer=True, img_size=320, counting=True)

video_mp4 = io.open(video + "/video.mp4", 'r+b').read()
encoded = base64.b64encode(video_mp4)
HTML(data='''<video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii')))